In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from feature_engine.outliers import Winsorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [2]:
df_missing = pd.read_csv('../Data/df_transformed.csv')
df_missing.drop('Unnamed: 0',axis=1,inplace=True)
df_imputed = pd.read_csv('../Data/dfi_transformed.csv')
df_imputed.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
Models = {
    'number':[],
    'description':[],
    'r2':[],
    'adjusted r2':[],
    'training MAE':[],
    'testing MAE':[],
    'training RMSE':[],
    'testing RMSE':[]
}

In [4]:
def model(number,return_model=False):
    global Models,df_missing,df_imputed
    original_number = number
    m = ""
    imputed = False
    num_variables=['RIDAGEYR','BPXPLS','Systolic','Diastolic']
    cat_variables =['RIAGENDR','RIDRETH3']
    if number % 2 == 0: # use imputed values (remember to drop na before train test split)
        df = df_missing
        m = "drop missing, "
    else:
        df = df_imputed
        imputed = True
        m = "imputed, "
        cat_variables.extend(['BPXPLS_i','Systolic_i','Diastolic_i'])
    number = (number - number%2)//2
    if number%2==1: # drop outliers
        m = m + "drop outliers, "
        df = df[df.outlier==0].copy()
    number = (number - number%2)//2
    if number % 2 == 1: # reduce y range
        m = m + "reduce y range, "
        number -= 1
        df = df[(df.LBXTC>=100)&(df.LBXTC<400)].copy()
    number = number//2
    if number % 2 == 1: # use height and weight
        m = m + "use height and weight, "
        number = number - 1
        num_variables.extend([ 'BMXWT', 'BMXHT'])
        if imputed:
            cat_variables.extend(['BMXWT_i', 'BMXHT_i'])
    else:
        m = m + "use bmi, "
        num_variables.append('BMXBMI')
        if imputed:
            cat_variables.append('BMXBMI_i')
    number = number//2
    if number%2==1: # include irregular pulse
        m = m + "include irregular pulse, "
        number -= 1
        cat_variables.append('BPXPULS')
        if imputed:
            cat_variables.append('BPXPULS_i')
    number = number//2
    if number % 2 == 1: # include chronic conditions
        number -= 1
        m = m + "include chronic conditions, "
        cat_variables.extend(['BPQ020', 'BPQ050A', 'BPQ080','BPQ100D', 'DIQ010', 'MCQ170M'])
        if imputed:
            cat_variables.extend(['BPQ020_i', 'BPQ050A_i', 'BPQ080_i', 'BPQ100D_i', 'DIQ010_i','MCQ170M_i'])
    number = number//2
    if number % 2 == 1: # use transformed physical activity
        number -= 1
        m = m + "transformed physical activity,"
        num_variables.append('MET_rec')
        if imputed:
            cat_variables.append('MET_rec_i')
    else:
        num_variables.extend(['PAQ655','PAD660','PAQ670','PAD675'])
        if imputed:
            cat_variables.extend(['PAQ655_i','PAD660_i','PAQ670_i','PAD675_i'])
    number = number // 2
    if number%2 == 0: # don't include nutrients
        number = (number - number%6)//6
    else:
        number = (number-1)//2
        if number % 3 == 0: # include original nutrients
            num_variables.extend(['DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA'])
            cat_variables.append('DR1_300')
            if imputed:
                cat_variables.append('DR1_300_i')
            m = m + "original nutrients included, "
        elif number % 3 == 1: # include transformed nutrients
            m = m + "transformed nutrients included"
            num_variables.extend(['DR1TKCAL_t',
       'DR1TPROT_t', 'DR1TCARB_t', 'DR1TSUGR_t', 'DR1TFIBE_t', 'DR1TTFAT_t',
       'DR1TSFAT_t', 'DR1TMFAT_t', 'DR1TPFAT_t', 'DR1TCHOL_t', 'DR1TSODI_t',
       'DR1TVD_t', 'DR1TCALC_t', 'DR1TIRON_t', 'DR1TPOTA_t'])
            if imputed:
                cat_variables.append('DR1_300_i')
        else: # include both
            m = m + "both nutrients included, "
            num_variables.extend(['DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA','DR1TKCAL_t',
       'DR1TPROT_t', 'DR1TCARB_t', 'DR1TSUGR_t', 'DR1TFIBE_t', 'DR1TTFAT_t',
       'DR1TSFAT_t', 'DR1TMFAT_t', 'DR1TPFAT_t', 'DR1TCHOL_t', 'DR1TSODI_t',
       'DR1TVD_t', 'DR1TCALC_t', 'DR1TIRON_t', 'DR1TPOTA_t'])
            if imputed:
                cat_variables.append('DR1_300_i')
        number = (number - number%3)//3
    if number % 2 == 0: # don't include alcohol and tobacco
        number = (number - number % 6)//6
    else: # include alcohol and tobacco
        number = (number - 1)//2
        # add tobacco here
        cat_variables.append('SMQ681')
        if imputed:
            cat_variables.append('SMQ681_i')
        if number % 3 == 0: # include original alcohol quantities
            m = m + "tobacco and alcohol questionairre variables"
            num_variables.extend([ 'ALQ120Q','ALQ130'])
            cat_variables.append('ALQ120U')
            if imputed:
                cat_variables.extend(['ALQ120Q_i', 'ALQ120U_i', 'ALQ130_i'])
        elif number % 3 == 1: # include transformed alcohol quantities
            m = m + "tobacco and average alcohol, "
            num_variables.append('avgALC')
            if imputed:
                cat_variables.append('avgALC_i')
        else: # include both
            m = m + "tobacco and both alcohol variables"
            num_variables.extend([ 'ALQ120Q','ALQ130','avgALC'])
            cat_variables.append('ALQ120U')
            if imputed:
                cat_variables.extend(['ALQ120Q_i', 'ALQ120U_i', 'ALQ130_i','avgALC_i'])
        number = (number-number%3)//3
    df = df[num_variables+cat_variables+['LBXTC','log_LBXTC']].copy()
    df.dropna(inplace=True)
    y = df.LBXTC
    if number % 2 == 1: # use log transformed y
        m = m + "log transform y"
        number -= 1
        y = df.log_LBXTC
    number = number//2
    X = df[num_variables+cat_variables].copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=675, test_size=0.2)
    numeric_transformer = Pipeline(steps=[('scalar',StandardScaler())])
    numeric_transformer.steps.pop(0)
    cat_transformer = Pipeline(steps=[('one_hot',OneHotEncoder(handle_unknown='ignore'))])
    if number % 2 == 1: # Winsorize x
        m = m + "Winsorize x, "
        numeric_transformer.steps.insert(0,('winsorizor',Winsorizer()))
        number -= 1
    number = number//2
    if number % 2 == 1: # log transform x
        m = m + "Log transform x values, "
        number -= 1
        numeric_transformer.steps.insert(0,('log transform x',FunctionTransformer(np.log1p)))
    number = number // 2
    if number%2==1: # add a scaler
        number -= 1
        m = m + "scale values, "
        numeric_transformer.steps.append(('scalar',StandardScaler()))
    number = number//2
    preprocessor = ColumnTransformer(transformers=[
        ('num',numeric_transformer,num_variables),
        ('cat',cat_transformer,cat_variables)])
    rf1 = Pipeline(steps =[
        ('preprocessor',preprocessor),
        ('rf', RandomForestRegressor())])
    if number % 2 == 1: # add PCA
        rf1.steps.insert(1,("pca",PCA(0.95)))
        m = m + "added PCA"

    rf1.fit(X_train,y_train)
    if return_model:
        return (rf1, X_train, X_test, y_train, y_test)
    Models['number'].append(original_number)
    Models['description'].append(m)
    r2 = rf1.score(X_train,y_train)
    Models['r2'].append(r2)
    k = rf1.named_steps['rf'].n_features_in_
    #n = rf1.named_steps['rf'].n_samples_fit_
    # may need to fix this as no n samples fit and pca may reduce features
    n = X_train.shape[0]
    ar2 = ((1-r2)/(n-1))/(n-k-1)
    Models['adjusted r2'].append(ar2)
    y_train_preds = rf1.predict(X_train)
    y_preds = rf1.predict(X_test)
    Models['training MAE'].append(mean_absolute_error(y_train,y_train_preds))
    Models['testing MAE'].append(mean_absolute_error(y_test,y_preds))
    Models['training RMSE'].append(mean_squared_error(y_train,y_train_preds,squared=False))
    Models['testing RMSE'].append(mean_squared_error(y_test,y_preds,squared=False))

In [5]:
model_numbers = []
for i in range(20):
    n = np.random.randint(2*73728)
    while n in model_numbers:
        n = np.random.randint(2*73728)
    model_numbers.append(n)
    try:
        model(n)
    except:
        # the error might be that the numeric transformer has no steps
        print(n)
models = pd.DataFrame(Models)
models.sort_values(by='adjusted r2',ascending=False)

5543
8790
5708
7332


,number,description,r2,adjusted r2,training MAE,testing MAE,training RMSE,testing RMSE
3,53140,"drop missing, reduce y range, use bmi, include...",0.872997,1.574698e-08,0.058962,0.157114,0.075040,0.197176
2,66984,"drop missing, use height and weight, include c...",0.882566,1.484199e-08,10.868637,28.547384,14.382428,37.626857
0,37204,"drop missing, reduce y range, use bmi, include...",0.868231,1.021275e-08,11.382441,30.643296,14.631361,38.539951
13,32716,"drop missing, reduce y range, use height and w...",0.872140,1.002678e-08,0.059353,0.161819,0.075513,0.204041
7,43277,"imputed, reduce y range, use height and weight...",0.868656,9.735938e-09,0.060571,0.161814,0.076740,0.204057
4,52910,"drop missing, drop outliers, reduce y range, u...",0.883337,9.548743e-09,0.056492,0.153319,0.071959,0.195439
11,72451,"imputed, drop outliers, use bmi, log transform...",0.870224,9.541611e-09,0.061742,0.167229,0.079314,0.214261
12,27696,"drop missing, use bmi, include irregular pulse...",0.879553,9.525347e-09,11.046493,29.315548,14.634040,37.778383
14,51085,"imputed, reduce y range, use height and weight...",0.872116,9.523802e-09,0.059599,0.159653,0.075722,0.201683
10,63584,"drop missing, use bmi, include chronic conditi...",0.879642,9.504861e-09,0.059033,0.155715,0.076400,0.197375


In [6]:
Models

{'number': [37204,
  27705,
  66984,
  53140,
  52910,
  13815,
  9843,
  43277,
  22048,
  52609,
  63584,
  72451,
  27696,
  32716,
  51085,
  52277],
 'description': ['drop missing, reduce y range, use bmi, include irregular pulse, transformed physical activity,scale values, ',
  'imputed, use height and weight, include irregular pulse, include chronic conditions, Winsorize x, Log transform x values, ',
  'drop missing, use height and weight, include chronic conditions, original nutrients included, tobacco and average alcohol, Winsorize x, Log transform x values, scale values, ',
  'drop missing, reduce y range, use bmi, include irregular pulse, original nutrients included, tobacco and average alcohol, log transform yWinsorize x, scale values, ',
  'drop missing, drop outliers, reduce y range, use height and weight, include chronic conditions, both nutrients included, log transform yWinsorize x, scale values, ',
  'imputed, drop outliers, reduce y range, use bmi, include irregular 

In [7]:
model_numbers

[5543,
 8790,
 37204,
 5708,
 27705,
 66984,
 53140,
 52910,
 13815,
 9843,
 43277,
 22048,
 52609,
 63584,
 7332,
 72451,
 27696,
 32716,
 51085,
 52277]